In [12]:
import torch
import torch.nn as nn


In [13]:
from CNN_Mask_to_Tens import CNN

NameError: name 'nn' is not defined

In [ ]:
# Load model
model = CNN()

# Load the trained weights
model.load_state_dict(torch.load('trained_model'))
# Set the model in evaluation mode

model.eval()

# Make a prediction
with torch.no_grad():
    output = model(input_tensor)

# Convert the output to a readable format
predicted_value = output.item()

# Print the predicted value
print('Predicted value:', predicted_value)